# Lab 7: Human motion generation

## Advanced deep learning

## Setup

### Setup dataset


Please access this Google Drive folder: [link](https://drive.google.com/drive/folders/1V5yzlwBPSNVPj33SfDHnvMykXISh3CyB?usp=sharing) and create a shortcut in the root of your Google Drive `/content/drive/MyDrive/`.

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!ls /content/drive/MyDrive/humanml3d-data

Mounted at /content/drive
caption_clip  caption_raw  checkpoints	humanml3d_test_split.txt  smplh  smpl_rifke


### Setup environment

Make sure you're running on a T4 GPU Colab instance; if not, activate it.

In [ ]:
!nvidia-smi

In [2]:
import torch
torch.cuda.is_available()

True

Clone the lab repository.

In [3]:
!git clone https://github.com/Leon-Chin/DDPM_DDIM_DIT.git
# !git clone https://github.com/robincourant/lab-MotionDiT.git


Cloning into 'DDPM_DDIM_DIT'...
remote: Enumerating objects: 78, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 78 (delta 5), reused 78 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (78/78), 1.64 MiB | 19.97 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [4]:
# %cd lab5-CSC52087EP
%cd lab-MotionDiT
!ln -s /content/drive/MyDrive/humanml3d-data ./ # Plug the dataset in the repo

/content/lab-MotionDiT


Install required libraries

In [5]:
!pip install hydra-core
!pip install pyrender
!pip install smplx
!pip install torchtyping
!pip install lightning
!pip install ema_pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.9 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144555 sha256=a20c0988a2b4ec15cc79c287533322acd2a060c8a5e4684bbba5b1ea68587ab8
  Stored in directory: /root/.cache/pip/wheels/1a/97/32/461f837398029ad76911109f07047fde1d7b661a147c7c56d1
Successfully built antlr4-python3-runtime


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 41.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 962.1/962.1 kB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 708.6/708.6 kB 42.6 MB/s eta 0:00:00
  Created wheel for PyOpenGL: filename=PyOpenGL-3.1.0-py3-none-any.whl size=1745193 sha256=e45aab272ccb01a180e9e62cfacda0b248fe47803339a6f954c59a694a0ef48f
  Stored in directory: /root/.cache/pip/wheels/2f/37/f5/f88cd3dddf75bc3ce608e44bf8a79078c408bf1f351a50818e
Successfully built PyOpenGL
  Attempting uninstall: PyOpenGL
    Found existing installation: PyOpenGL 3.1.9
    Uninstalling PyOpenGL-3.1.9:
      Successfully uninstalled PyOpenGL-3.1.9
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Human motion dataset and representation

### HumanML3D dataset

In [8]:
import os

data_path = './humanml3d-data'

# 查看数据目录下内容
print(os.listdir(data_path))


['humanml3d_test_split.txt', 'caption_clip', 'smpl_rifke', 'caption_raw', 'smplh', 'checkpoints']


In [9]:
import numpy as np
import glob

motion_files = glob.glob(os.path.join(data_path, 'smpl_rifke', '*.npy'))
num_motions = len(motion_files)

motion_lengths = [np.load(motion).shape[0] for motion in motion_files]
average_motion_length = np.mean(motion_lengths)

description_files = glob.glob(os.path.join(data_path, 'caption_raw', '*.txt'))
num_descriptions = len(description_files)

description_lengths = []
for desc_file in description_files:
    with open(desc_file, 'r') as f:
        description = f.read().strip()
        description_lengths.append(len(description.split()))

average_description_length = np.mean(description_lengths)

print(f"Total number of motions: {num_motions}")
print(f"Average motion length (frames): {average_motion_length:.2f}")

print(f"Total number of descriptions: {num_descriptions}")
print(f"Average description length (words): {average_description_length:.2f}")


Total number of motions: 3893
Average motion length (frames): 143.92
Total number of descriptions: 3893
Average description length (words): 37.28


#### Question 1:
*Answer here...*

### SMPL representation


#### Question 2:
*Answer here...*

#### Code 2
*Complete `visualize_smpl.py`*

In [10]:
!HYDRA_FULL_ERROR=1 PYTHONPATH=$(pwd) python src/visualize_smpl.py

import moviepy.editor
moviepy.editor.ipython_display("./smpl.mp4")

[2025-02-28 22:06:03,855][numexpr.utils][INFO] - NumExpr defaulting to 2 threads.
/content/lab-MotionDiT/src/datasets/modalities/human_dataset.py:44: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this e

  if event.key is 'enter':



OSError: MoviePy error: the file ./smpl.mp4 could not be found!
Please check that you entered the correct path.

## Model architectures

### Config A: Incontext

#### Code 3
*Complete `src/models/modules/incontext.py`*

In [ ]:
!PYTHONPATH=$(pwd) python src/models/modules/incontext.py

### Config B: AdaLN

#### Question 3:
*Answer here...*

#### Code 4
*Complete `src/models/modules/adaln.py`*

In [ ]:
!PYTHONPATH=$(pwd) python src/models/modules/adaln.py

### Config C: Cross attention

#### Question 4:
*Answer here...*

#### Code 5
*Complete `src/models/modules/cross attention.py`*

In [ ]:
!PYTHONPATH=$(pwd) python src/models/modules/cross_attention.py

#### Question 5:
*Answer here...*

## Diffusion framework

### DDPM

#### Code 6
*Complete `src/training/losses/ddpm.py`*

In [ ]:
!PYTHONPATH=$(pwd) python src/training/losses/ddpm.py

#### Code 7
*Complete `src/training/sampler/ddpm.py`*

In [ ]:
!PYTHONPATH=$(pwd) python src/generate.py batch_size=1 diffuser/sampler@diffuser.test_sampler=ddpm seed=2

import moviepy.editor
moviepy.editor.ipython_display("./generation_ddpm_incontext.mp4")

### DDIM

#### Question 6:
*Answer here...*

#### Code 8
*Complete `src/training/sampler/ddim.py`*

In [ ]:
!PYTHONPATH=$(pwd) python src/generate.py batch_size=1 diffuser/sampler@diffuser.test_sampler=ddim seed=2

import moviepy.editor
moviepy.editor.ipython_display("./generation_ddim_incontext.mp4")

#### Question 7:
*Answer here...*

## Result analysis

### Qualitative analysis

#### Code 9

In [ ]:
!PYTHONPATH=$(pwd) python src/generate.py batch_size=1 diffuser/network=incontext \
checkpoint_path=./humanml3d-data/checkpoints/incontext.ckpt

import moviepy.editor
moviepy.editor.ipython_display("./generation_ddpm_incontext.mp4")

In [ ]:
!PYTHONPATH=$(pwd) python src/generate.py batch_size=1 diffuser/network=adaln \
checkpoint_path=./humanml3d-data/checkpoints/adaln.ckpt

import moviepy.editor
moviepy.editor.ipython_display("./generation_ddpm_adaln.mp4")

In [ ]:
!PYTHONPATH=$(pwd) python src/generate.py batch_size=1 diffuser/network=cross_attention \
checkpoint_path=./humanml3d-data/checkpoints/cross_attention.ckpt

import moviepy.editor
moviepy.editor.ipython_display("./generation_ddpm_cross_attention.mp4")

#### Question 8
*Answer here...*

### Quantitative analysis

#### Question 9:
*Answer here...*

#### Code 10
*Complete src/metrics/frechet.py*

#### Code 11
*Complete src/metrics/similarity.py*

#### Code 12

#### Bonus 1:
*Answer here...*

In [ ]:
!PYTHONPATH=$(pwd) python src/evaluate.py diffuser/network=incontext \
checkpoint_path=./humanml3d-data/checkpoints/incontext.ckpt

In [ ]:
!PYTHONPATH=$(pwd) python src/evaluate.py diffuser/network=adaln \
checkpoint_path=./humanml3d-data/checkpoints/adaln.ckpt

In [ ]:
!PYTHONPATH=$(pwd) python src/evaluate.py diffuser/network=cross_attention \
checkpoint_path=./humanml3d-data/checkpoints/cross_attention.ckpt

#### Question 10:
*Answer here...*

#### Bonus 2:
*Answer here...*